In [ ]:
import tensorflow as tf
import json
import logging
import pickle

In [ ]:
# 读文件进行NER
# 文件以行为单位，每一行包含上下文关系密切的一段话

def load_config(config_file):
    with open(config_file, encoding="utf8") as f:
        return json.load(f)

def get_logger(log_file):
    logger = logging.getLogger(log_file)
    logger.setLevel(logging.DEBUG)
    fh = logging.FileHandler(log_file)
    fh.setLevel(logging.DEBUG)
    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)
    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    ch.setFormatter(formatter)
    fh.setFormatter(formatter)
    logger.addHandler(ch)
    logger.addHandler(fh)
    return logger
    
config = load_config("ChineseNER/config_file")
logger = get_logger("ChineseNER/log/train.log")

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
with open("ChineseNER/maps.pkl", "rb") as f:
    char_to_id, id_to_char, tag_to_id, id_to_tag = pickle.load(f)

In [ ]:
with tf.Session(config=tf_config) as sess:
    model = create_model(sess, Model, FLAGS.ckpt_path, load_word2vec, config, id_to_char, logger)
    with codecs.open("ChineseNER/dataset/") as f:
        line = input("请输入测试句子:")
        result = model.evaluate_line(sess, input_from_line(line, char_to_id), id_to_tag)
        print(result)

In [ ]:
import mysql.connector as c

db = c.connect(host="localhost", user="root", passwd="123456", db="news_spider")
cur = db.cursor()
cur.execute("SELECT id, title, newstype, time_pub, content FROM t_people where newstype=\"'military'\" order by time_pub desc")

try:
    result = cur.fetchmany(size=100)
    while result:
        for (id, title, newstype, time_pub, content) in result:
            print(time_pub + '\t' + title)
        break
except Exception as e:
    print(e)
finally:
    db.close()

In [ ]:
!wget "https://bosonnlp.com/resources/BosonNLP_NER_6C.zip" "ChineseNER/data/"
!unzip "BosonNLP_NER_6C.zip"

In [ ]:
import codecs

"""
格式转换：
源格式的标记类型为：
product_name, time, person_name, org_name, location, company_name

目标格式中的标记格式为M-TYPE
M取值为B或I，分别表示开始和其他
TYPE取值分别为LOC, PER, ORG
"""

tag_dict = {'product_name': 'PDT',
           'time': 'T',
           'person_name':'PER',
           'org_name':'ORG',
           'company_name':'COM',
           'location':'LOC'}
spaces = {' ', '\xa0', '\u3000', '\t'}

with codecs.open("ChineseNER/data/BosonNLP_NER_6C.txt", "r", "utf8") as rf:
    with codecs.open("ChineseNER/data/BosonNLP_NER_6C.train", "w", "utf8") as wf:
        for n, line in enumerate(rf.readlines()):
            line = line.replace("\\n", "")
            print(str(n))
            flag = 0
            while flag < len(line):
                if line[flag] == '{':
                    begin = flag
                    while line[flag] != '}':
                        flag += 1
                    tags = line[begin+2 : flag]
                    for space in spaces:
                        tags = tags.replace(space, "")
                    tags = tags.split(":")
                    new_item = tags[1][0] + ' B-'+ tag_dict[tags[0]]
                    wf.write(new_item + '\n')
                    for a in tags[1][1:]:
                        new_item = a + ' I-'+tag_dict[tags[0]]
                        wf.write(new_item + '\n')
                    flag += 2
                else:
                    if line[flag] != '\n':
                        if line[flag] not in spaces:
                            new_item = line[flag] + ' O'
                            wf.write(new_item + '\n')
                        if line[flag] == '。':
                            wf.write('\n')
                    flag += 1
    